In [1]:
import numpy as np
from numpy import array
from PIL import Image
import PIL
from skimage import io,color
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import numpy as np
from numpy import array
import os
from sklearn_theano.datasets import fetch_asirra
from sklearn_theano.feature_extraction import OverfeatTransformer
%matplotlib inline
os.chdir('/Users/RayLJazz/Dropbox/UCDavis/2016summer/DSG/')

/Users/RayLJazz/miniconda3/envs/python2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
### read training data
trainPath = "id_train.csv"
train = pd.read_csv(trainPath) # 8000
all_train_id = train['Id']
all_labels = train['label']
### get testing data id
testPath = "sample_submission4.csv"
test = pd.read_csv(testPath) # 13998
test_id = test["Id"]
### read all images'id into array
path = "roof_images/"
pics = os.listdir(path) # 42760 pictures

In [3]:
len(pics)

42760

In [4]:
pics.remove('.DS_Store')

In [5]:
len(pics)

42759

In [6]:
train['label'].value_counts()

1    3479
2    1856
4    1806
3     859
Name: label, dtype: int64

We will use stratified cross validation.

## Resize image function

In [139]:
def resizeImg(imgId, stand_size=(256, 256)):
    # stand_size if the size an image is scaled to
    # e.g stand_size = (256, 256)
    fromDir = 'roof_images'
    toDir = 'resized_images'
    imgFile = fromDir+'/'+str(imgId)
    img = Image.open(imgFile)
    img = img.resize(stand_size, PIL.Image.ANTIALIAS)
    img.save(toDir+'/'+str(imgId))

In [140]:
resize = lambda i: resizeImg(i)
resize_color_origin = pd.Series(data=pics).apply(resizeImg)

In [160]:
all_train_id.shape

(8000,)

In [158]:
test_id.shape

(13999,)

In [161]:
def resizeImg_array(imgId, stand_size=(256, 256)):
    fromDir = 'resized_images'
    imgFile = fromDir+'/'+str(imgId)+'.jpg'
    img = io.imread(imgFile)
    return img

In [142]:
pics[0:2]

['-1166002.jpg', '-1191173.jpg']

In [175]:
resize = lambda i: resizeImg_array(i)
resize_train = pd.Series(data=all_train_id).apply(resize)
resize_color_origin_train=pd.DataFrame()
resize_color_origin_train['Id']=all_train_id
resize_color_origin_train['ColorArray']=resize_train

In [176]:
resize_color_origin_train.head()

,Id,ColorArray
0,-3935637,"[[[29, 44, 41], [27, 42, 39], [23, 38, 35], [1..."
1,-2815459,"[[[75, 89, 92], [81, 92, 98], [85, 96, 102], [..."
2,-2355684,"[[[40, 49, 46], [38, 47, 44], [36, 45, 42], [3..."
3,-2343303,"[[[27, 48, 67], [26, 47, 64], [26, 47, 64], [2..."
4,-1981985,"[[[123, 127, 126], [122, 126, 125], [118, 122,..."


In [179]:
resize_color_origin_train.shape

(8000, 2)

In [177]:
import pickle
with open('resize_color_origin_train.pickle', 'wb') as handle:
    pickle.dump(resize_color_origin_train, handle,protocol=2)

In [178]:
resize = lambda i: resizeImg_array(i)
resize_test = pd.Series(data=test_id).apply(resize)
resize_color_origin_test=pd.DataFrame()
resize_color_origin_test['Id']=test_id
resize_color_origin_test['ColorArray']=resize_test
resize_color_origin_test.shape

(13999, 2)

In [37]:
resize_color_origin_test['ColorArray'][0].shape

(256, 256, 3)

In [181]:
import pickle
with open('resize_color_origin_test.pickle', 'wb') as handle:
    pickle.dump(resize_color_origin_test, handle,protocol=2)

In [7]:
#resize_color_origin_train=pd.read_pickle('resize_color_origin_train.pickle')
resize_color_origin_test=pd.read_pickle('resize_color_origin_test.pickle')

In [8]:
origin_train_Matrix=[]
test = resize_color_origin_train['ColorArray'].apply(origin_train_Matrix.append)
origin_train_Matrix=np.array(origin_train_Matrix)

In [9]:
origin_train_Matrix[0]

array([[[29, 44, 41],
        [27, 42, 39],
        [23, 38, 35],
        ..., 
        [19, 29, 20],
        [ 9, 19, 10],
        [20, 30, 21]],

       [[26, 41, 38],
        [25, 40, 37],
        [24, 39, 36],
        ..., 
        [19, 29, 20],
        [ 9, 19, 10],
        [20, 30, 21]],

       [[18, 33, 30],
        [20, 35, 32],
        [22, 37, 34],
        ..., 
        [19, 29, 20],
        [ 9, 19, 10],
        [20, 30, 21]],

       ..., 
       [[20, 27, 20],
        [23, 30, 23],
        [25, 32, 25],
        ..., 
        [13, 28, 25],
        [20, 34, 34],
        [27, 41, 41]],

       [[26, 31, 25],
        [29, 34, 28],
        [30, 35, 29],
        ..., 
        [21, 36, 33],
        [30, 44, 44],
        [39, 53, 53]],

       [[35, 40, 34],
        [36, 41, 35],
        [37, 42, 36],
        ..., 
        [32, 47, 44],
        [41, 55, 55],
        [51, 65, 65]]], dtype=uint8)

In [10]:
origin_train_Matrix[0].shape

(256, 256, 3)

In [9]:
origin_train_Matrix.shape

(8000, 256, 256, 3)

In [12]:
import pickle
with open('origin_train_Matrix.pickle', 'wb') as handle:
    pickle.dump(origin_train_Matrix, handle,protocol=2)

In [8]:
origin_train_Matrix=pd.read_pickle('origin_train_Matrix.pickle')

In [8]:
origin_test_Matrix=[]
test = resize_color_origin_test['ColorArray'].apply(origin_test_Matrix.append)
origin_test_Matrix=np.array(origin_test_Matrix)

In [9]:
origin_test_Matrix[0].shape

(256, 256, 3)

In [10]:
origin_test_Matrix.shape

(13999, 256, 256, 3)

In [14]:
origin_test_Matrix[8000:13999].shape

(5999, 256, 256, 3)

In [11]:
del resize_color_origin_test

In [19]:
#import pickle
#with open('origin_test_Matrix.pickle', 'wb') as handle:
#    pickle.dump(origin_test_Matrix, handle,protocol=2)

# Extract CNN layers

In [12]:
def CNN_feature(layer,X):
    tf = OverfeatTransformer(output_layers=[layer])
    X_tf = tf.transform(X)
    X_tf.shape
    return X_tf

In [14]:
del origin_train_Matrix

In [11]:
CNN_4 = CNN_feature(4,origin_train_Matrix)

In [13]:
CNN_4.shape

(8000, 43264)

In [16]:
#import pickle
#with open('CNN_4_train.pickle', 'wb') as handle:
#    pickle.dump(CNN_4, handle,protocol=2)

In [7]:
CNN_4=pd.read_pickle('CNN_4_train.pickle')

In [18]:
train = pd.DataFrame(CNN_4)
train['Class'] = all_labels
train['Id'] = all_train_id
target = 'Class'
Predictors=[x for x in train.columns if x not in ['Id','Class']]

In [8]:
del CNN_4

In [24]:
train['Class']=train['Class']-1

In [25]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,43256,43257,43258,43259,43260,43261,43262,43263,Class,Id
0,0.000000,0.000000,1.084932,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.804425,0.000000,0.000000,0.000000,3.459209,1.814213,0.294278,3.943680,3,-3935637
1,0.000000,0.686480,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.103195,5.088358,3.797582,0.000000,0.000000,0.000000,8.738600,3.650664,2,-2815459
2,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.178296,2.207493,1.358124,2.292116,2.343516,2.346575,2.134747,3.390357,2,-2355684
3,6.458548,0.000000,4.098860,5.11187,0.0,0.0,0.0,0.0,0.0,4.961978,...,2.290351,6.531635,9.873084,2.999096,1.514193,2.112572,2.616589,4.357081,3,-2343303
4,0.000000,11.419174,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.314942,1.754873,2.425998,2.852057,4.324605,3.768018,2.952718,2.096267,1,-1981985


In [28]:
train.shape

(8000, 43266)

In [11]:
del train

In [29]:
with open('CNN_4_train.pickle', 'wb') as handle:
    pickle.dump(train, handle,protocol=2)

In [9]:
train=pd.read_pickle('CNN_4_train.pickle')

In [10]:
train.shape

(8000, 43266)

In [15]:
CNN_4_test1 = CNN_feature(4,origin_test_Matrix[0:8000])

In [16]:
CNN_4_test1.shape

(8000, 43264)

In [18]:
test1 = pd.DataFrame(CNN_4_test1)
test1['Id'] = test_id[0:8000]
Predictors_test=[x for x in test1.columns if x not in ['Id']]

In [19]:
test1.shape

(8000, 43265)

In [21]:
import pickle
with open('CNN_4_test1.pickle', 'wb') as handle:
    pickle.dump(test1, handle,protocol=2)

In [22]:
CNN_4_test2 = CNN_feature(4,origin_test_Matrix[8000:13999])

In [23]:
CNN_4_test2.shape

(5999, 43264)

In [24]:
test2 = pd.DataFrame(CNN_4_test2)
test2['Id'] = test_id[8000:13999]
#Predictors_test=[x for x in test1.columns if x not in ['Id']]

In [25]:
test2['Id'] = test_id[8000:13999]

In [26]:
test2.shape

(5999, 43265)

In [27]:
import pickle
with open('CNN_4_test2.pickle', 'wb') as handle:
    pickle.dump(test2, handle,protocol=2)

In [3]:
test1 = pd.read_pickle('CNN_4_test1.pickle')
test2 = pd.read_pickle('CNN_4_test2.pickle')

In [4]:
df = [test1,test2]
test_pd = pd.concat(df)

In [5]:
test_pd.shape

(13999, 43265)

In [6]:
del test1

In [7]:
del test2

In [8]:
#import pickle
#with open('test_pd.pickle', 'wb') as handle:
#    pickle.dump(test_pd, handle,protocol=2)

In [8]:
test_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,43255,43256,43257,43258,43259,43260,43261,43262,43263,Id
0,0.000000,0.00000,0.0,0.000000,0.000000,3.478970,2.191785,0.000000,0.000000,0.000000,...,4.926783,5.279008,6.531221,6.964129,5.312031,3.893291,4.606564,2.955109,2.401158,766723500.0
1,0.000000,0.00000,0.0,0.207979,0.000000,3.282540,0.000000,0.000000,0.000000,0.000000,...,2.794873,4.657119,1.750573,0.000000,0.000000,0.000000,0.000000,6.048339,6.132316,739273600.0
2,1.219605,2.01839,0.0,1.710595,3.242033,0.000000,3.001603,1.737243,0.097029,2.060579,...,6.416501,10.445930,5.577863,0.000000,0.000000,2.792330,5.694457,6.024070,2.330191,156621880.0
3,0.000000,0.00000,0.0,0.000000,1.040887,1.131448,0.000000,0.000000,0.000000,0.000000,...,1.757758,1.770751,3.008193,2.760686,0.941267,2.582855,0.000000,0.477683,0.279537,156551275.0
4,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,1.828242,0.000000,0.000000,...,0.600482,1.571823,1.689718,1.442909,0.000000,1.170244,2.912273,2.826955,5.088355,793980800.0


## Extreme Gradient Boosting

### Step 1- Find the number of estimators for a high learning rate

In [9]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [22]:
def modelfit(alg, dtrain, predictors,target, useTrainCV=True, cv_folds=5, early_stopping_round=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        #xgtest = xgb.DMatrix(dtest[predictors].values)
        xgb_param_forcv = xgb_param
        xgb_param_forcv['num_class'] = 4
        cvresult = xgb.cv(xgb_param_forcv, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_round, show_progress=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='merror')
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    #dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    return alg, dtrain_predictions

In [26]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=100,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'multi:softmax',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1, dtrain_predictions = modelfit(xgb1, train, Predictors,target)

Will train until cv error hasn't decreased in 50 rounds.
[0]	cv-test-merror:0.41+0.0114291294507	cv-train-merror:0.28525+0.00404279452854
[1]	cv-test-merror:0.366375+0.0114455231423	cv-train-merror:0.2295938+0.00519566316845
[2]	cv-test-merror:0.350125+0.00764852927039	cv-train-merror:0.204+0.00296402914965
[3]	cv-test-merror:0.341375+0.00861865708797	cv-train-merror:0.1898438+0.00113975741279
[4]	cv-test-merror:0.33275+0.00967115556694	cv-train-merror:0.1781564+0.00367562368041
[5]	cv-test-merror:0.324+0.012409673646	cv-train-merror:0.1685626+0.00570978656694
[6]	cv-test-merror:0.32025+0.00686476510887	cv-train-merror:0.1620624+0.00346755813794
[7]	cv-test-merror:0.317875+0.0101365674664	cv-train-merror:0.1551876+0.00229106487032
[8]	cv-test-merror:0.315+0.0109330348028	cv-train-merror:0.1484062+0.00245351408392
[9]	cv-test-merror:0.309+0.00649037749287	cv-train-merror:0.143219+0.00250499189619


In [10]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=100,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'multi:softmax',
        nthread=4,
        scale_pos_weight=1,
        seed=27)

In [11]:
train=pd.read_pickle('CNN_4_train.pickle')

In [12]:
Predictors=[x for x in train.columns if x not in ['Id','Class']]

In [13]:
clf = xgb1.fit(train[Predictors],train['Class'],eval_metric='merror')

In [21]:
del train

In [15]:
Predictors_test=[x for x in test_pd.columns if x not in ['Id']]

In [16]:
test_result = xgb1.predict(test_pd[Predictors_test])

In [17]:
test_resultout = test_result+1
pd.Series(test_resultout).value_counts()

1    4114
2    3929
4    3067
3    2889
dtype: int64

In [18]:
test_resultout

array([4, 4, 1, ..., 3, 3, 3])

In [19]:
d = {'Id' : test_pd['Id'], 'Label' : test_resultout}
df = pd.DataFrame(d)
df.to_csv('/Users/RayLJazz/Dropbox/UCDavis/2016summer/DSG/pred10.csv', index=False)